# Various stress tests to see if instance and services response adequately

## Inputs and Configuration

In [13]:
import os
import random
import requests
import time
from inspect import cleandoc
from dataclasses import dataclass

PAVICS_HOST = os.getenv("PAVICS_HOST", "pavics.ouranos.ca").rstrip("/")
if not PAVICS_HOST:
    raise ValueError("Cannot run test without a PAVICS_HOST value.")
print(f"PAVICS_HOST: [{PAVICS_HOST}]")

PAVICS_URL = f"https://{PAVICS_HOST}"
VERIFY_SSL = True if "DISABLE_VERIFY_SSL" not in os.environ else False
MAGPIE_URL = PAVICS_URL + "/magpie"
TWITCHER_PROXY = "/twitcher/ows/proxy"
TWITCHER_URL = PAVICS_URL + TWITCHER_PROXY

PAVICS_HOST: [host-140-69.rdext.crim.ca]


## Utilities

In [20]:
@dataclass
class StressTestResult:
    code: int = 200
    runs: int = 0
    max_errors: int = 0
    method: str = "GET"
    url: str = None
    request_args: dict = None
    status: int = 0  # see description of stress-test
    codes = []
    delta = []
    times = []

    @property
    def avg_time(self):
        return sum(self.times) / self.runs

    @property
    def min_time(self):
        return min(self.times)

    @property
    def max_time(self):
        return max(self.times)

    def __str__(self):
        columns = ["Index", "Codes", "Delta", "Times"]
        idx = len(str(self.runs))
        r = max(len(columns[0]), idx)
        w = 10
        header = "".join(f"{c:>{w if i else r}}" for i, c in enumerate(columns))
        offset = 16
        data = [f"{i:>{r+(offset if i else 0)}}{c:>{w}}{d:>{w-1}.3f}s{t:>{w-1}.3f}s"
                for i, (c, d, t) in enumerate(zip(self.codes, self.delta, self.times))]
        lines = "\n".join(data)
        return cleandoc(f"""
        Stress Test:
            Test:
                code: {self.code}
                runs: {self.runs}
                max-errors: {self.max_errors}
            Request:
                method: {self.method}
                url:    {self.url}
                args:   {self.request_args}
            Times:
                min: {self.min_time:.3f}s
                avg: {self.avg_time:.3f}s
                max: {self.max_time:.3f}s
            Results:
                {header}
                {lines}
        """)


def stress_test_requests(url, runs=100, code=200, method="GET", delays=True,
                         max_err_code=0, max_avg_time=None, **req_kwargs):
    """
    Executes the request for the number of demanded runs and validates the expected status is always returned.

    Outputs the results of each request and a summary of their execution time.
    If requested, also validates that all responses were returned on average faster than the maximum allowed time.

    :returns:
        - 0 for no error (success)
        - 1 for HTT error code failure
        - 2 for max-time failure
    """
    print(f"\nStress Test with [{runs}] calls to [{url}]")
    result = StressTestResult()
    result.runs = runs
    result.url = url
    result.method = method
    result.request_args = req_kwargs
    result.max_errors = max_err_code
    result.max_avg_time = max_avg_time
    result.codes = []
    result.times = []
    result.delta = [0.] + [float((random.randint(1, 100) / 1000) if delays else 0) for _ in range(1, runs)]
    char = len(str(runs))
    for i in range(runs):
        if not i % 10:
            print(f"Progress: {i:>{char}}/{runs}")
        start = time.perf_counter()
        resp = requests.request(method, url, **req_kwargs)
        result.times.append(time.perf_counter() - start)
        result.codes.append(resp.status_code)
        if i == runs:
            break
        if result.delta[i]:
            time.sleep(result.delta[i])
    print(f"Progress: {runs:>{char}}/{runs}")
    if max_avg_time and result.avg_time > max_avg_time:
        result.status = 2
    elif len([c for c in result.codes if c == code]) >= (runs - max_err_code):
        result.status = 0
    else:
        result.status = 1
    return result


## Tests

In [22]:
# NBVAL_IGNORE_OUTPUT


for bird in ["finch", "flyingpigeon", "raven"]:
    bird_url = f"{TWITCHER_URL}/{bird}/wps?service=wps&request=getcapabilities"
    expect_max_avg_time = 1
    expect_status_code = 200
    results = stress_test_requests(bird_url, runs=100, code=expect_status_code, max_avg_time=expect_max_avg_time)
    if results.status == 1:
        raise AssertionError(f"Detected non HTTP {expect_status_code} codes.\n{results!s}")
    if results.status == 2:
        raise AssertionError(f"Detected regression with long request time.\n"
                             f"Expected max-avg-time: ({expect_max_avg_time:.3f}s <= {results.time_max:.3f}s).\n"
                             f"{results!s}")
    assert results.status == 0, f"Undefined failure condition encountered.\n{results!s}"
    print(results)


Stress Test with [100] calls to [https://host-140-69.rdext.crim.ca/twitcher/ows/proxy/finch/wps?service=wps&request=getcapabilities]
Progress:   0/100
Progress:  10/100
Progress:  20/100
Progress:  30/100
Progress:  40/100
Progress:  50/100
Progress:  60/100
Progress:  70/100
Progress:  80/100
Progress:  90/100
Progress: 100/100
Stress Test:
    Test:
        code: 200
        runs: 100
        max-errors: 0
    Request:
        method: GET
        url:    https://host-140-69.rdext.crim.ca/twitcher/ows/proxy/finch/wps?service=wps&request=getcapabilities
        args:   {}
    Times:
        min: 0.306s
        avg: 0.500s
        max: 1.122s
    Results:
        Index     Codes     Delta     Times
            0       200    0.000s    0.451s
            1       200    0.066s    0.376s
            2       200    0.083s    0.373s
            3       200    0.050s    0.521s
            4       200    0.017s    0.381s
            5       200    0.062s    0.380s
            6       200    0